In [1]:
import cvxpy as cp
import numpy as np

In [2]:
# interest rate for Net Present Value calculations
rate = 4/100; # 4.0#
# data for generator dispatch problem
D = 365; # days in a year
Y = 20; # years simulated
T = 24; # hours simulated in each day
t= np.arange(T); # vector of time
d = 40 * np.array([
    0.4, 0.4, 0.4, 0.65,
    0.65, 0.6, 0.6, 0.65,
    0.8, 0.85, 0.8, 0.8,
    0.75, 0.6, 0.5, 0.3, 
    0.3, 0.4, 0.54, 0.6, 
    0.75, 0.7, 0.6, 0.5
]) # daily demand in MWh
n = 4; # number of dispatchable generators (2 hard coal and 2 OCGT plants) to choose from,
# in addition to a solar plant
solar_cap_fact = np.concatenate((
    np.zeros(6),
    np.sin(np.pi*np.arange(0, 13)/12),
    np.zeros(24-6-(np.arange(0,13).size))
))
# capacity factor of the potential solar plant = power_available/installed_capacity
# (To be used in Part II only: capacity factor of the potential wind plant = power_available/installed_capacity)
wind_cap_fact = np.array([
    0.3277, 0.2865, 0.3303, 0.3073, 0.2994, 0.3071,
    0.2980, 0.2988, 0.3149, 0.3141, 0.3142, 0.5*0.3067, 
    0.5*0.2879, 0.5*0.3072, 0.5*0.3163, 0.5*0.3049, 0.5*0.3103, 
    0.3914, 0.4008 , 0.3879, 0.3889, 0.3999, 0.4153, 0.3923
])
Pmax = np.array([10, 5, 10, 15]) 
# Maximum generator capacities in MW [hard coal, hard coal, OCGT, OCGT] due to land constraints
Pmax_solar = 40; # Maximum capacity in MW for solar plant due to land constraints
Pmin = np.zeros(4) # generator minimum capacities in MW
Pmin_solar = 0;
ramp_hard_coal = 0.015 # 1.5# [# Pnom/minute]
hourly_ramp_hard_coal = ramp_hard_coal*60; # [# Pnom/h]
ramp_ocgt = 0.08 # 1.5# [# Pnom/min]
hourly_ramp_ocgt = ramp_ocgt*60; # [# Pnom/h]
R = np.array([hourly_ramp_hard_coal, hourly_ramp_hard_coal, hourly_ramp_ocgt, hourly_ramp_ocgt]) * Pmax 
# [hourly_ramp_hard_coal*Pmax[0], hourly_ramp_hard_coal*Pmax[1], hourly_ramp_ocgt*Pmax[2], hourly_ramp_ocgt*Pmax[3]]; # ramp-rate limits [MWh]
# source:
# https://www.eia.gov/electricity/annual/html/epa_08_04.html
#
fuel_cost_per_MWh_2018 = np.array([25.4, 25.4, 27.35, 27.35]); # $/MWh, 2018
#
#fuel_cost_per_MWh_2019 = [24.28 24.28 23.11 23.11]; # $/MWh, 2019 This
#cost won't be used
# Investment costs (or capital costs per MW of built capacity). From https://atb.nrel.gov/
capital_cost_solar_plant_per_MW = 1200000; # dollars/MW, denoted as M_i in the pdf
capital_cost_wind_plant_per_MW = 1300000; # dollars/MW, denoted as M_i in the pdf
capital_cost_coal_plant = 4200000; # dollars/MW, denoted as M_i in the pdf
capital_cost_gas_plant = 2600000; # dollars/MW, denoted as M_i in the pdf
# Emission rates of pounds of CO_2 per kWh for each tech (https://www.eia.gov/tools/faqs/faq.php?id=74&t=11)
co2_coal = 2.30; # pounds per kWh
co2_gas = 0.97; # pounds per kWh
co2_solar = 0;
# Remember to convert these to pounds/MWh for consistency in your units
# co2_emissions = [];
## Solution
## Optimal capacity and hourly dispatch

In [3]:
# define variables
p = cp.Variable((n, T))
p_solar = cp.Variable((1, T))
installed_thermal_capacity = cp.Variable((n, 1))
installed_solar_capacity = cp.Variable(1);

# define constraints
constraints = [
    cp.transpose(installed_thermal_capacity) >= Pmin,
    installed_solar_capacity >= Pmin_solar,
    cp.transpose(installed_thermal_capacity) <= Pmax,
    installed_solar_capacity <= Pmax_solar,
    cp.sum(p, axis=0) + p_solar >= d,
    p >= 0,
    p_solar >= 0,
    p <= installed_thermal_capacity,
    p_solar <= solar_cap_fact * installed_solar_capacity,
    # cp.abs(cp.diff(p, axis=1)) <= R[:, np.newaxis]
]

# define costs
coal_investment_cost = cp.sum(capital_cost_gas_plant * installed_thermal_capacity[0:2])
gas_investment_cost = cp.sum(capital_cost_gas_plant * installed_thermal_capacity[2:n])
solar_investment_cost = cp.sum(capital_cost_solar_plant_per_MW * installed_solar_capacity)

fuel_costs = cp.sum(365 * cp.transpose(p) @ fuel_cost_per_MWh_2018)

NPV = fuel_costs/(1 + rate)

# solve
problem = cp.Problem(cp.Minimize(
    NPV + coal_investment_cost + gas_investment_cost + solar_investment_cost
), constraints)

problem.solve()

print(problem.value)
print(installed_solar_capacity.value)
print(installed_thermal_capacity.value)
# print(p.value)

89567585.49838331
[5.65685425]
[[9.99999999]
 [4.99999998]
 [7.38259411]
 [7.61740591]]


C:\Users\LW\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\reductions\solvers\solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


In [4]:
q = np.arange(n*T).reshape(n, T)
print(q)
print(np.sum(q, axis=0))
print(np.transpose(q) @ (fuel_cost_per_MWh_2018))
print(q < np.ones((n, 1)))
# r = np.arange(24).reshape(1,24)
# r < solar_cap_fact
s = np.arange(4).reshape(4,1)
print(s.T < Pmax)
print((q < s).shape)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71]
 [72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]]
[144 148 152 156 160 164 168 172 176 180 184 188 192 196 200 204 208 212
 216 220 224 228 232 236]
[3891.6 3997.1 4102.6 4208.1 4313.6 4419.1 4524.6 4630.1 4735.6 4841.1
 4946.6 5052.1 5157.6 5263.1 5368.6 5474.1 5579.6 5685.1 5790.6 5896.1
 6001.6 6107.1 6212.6 6318.1]
[[ True False False False False False False False False False False False
  False False False False False False False False False False False False]
 [False False False False False False False False False False False False
  False False False False False False False False False False False False]
 [False False False False False False False False False False False False
  False False False False False False False False False 

In [5]:
print(sum(Pmax))
print(d)


40
[16.  16.  16.  26.  26.  24.  24.  26.  32.  34.  32.  32.  30.  24.
 20.  12.  12.  16.  21.6 24.  30.  28.  24.  20. ]
